In [5]:
import pandas as pd
from salary.models import *
import datetime
from daily.models import DeliveryStatus, NonDelivery, DeliveryBoy
from cash_collection.models import *
import git
import itertools as it
from collections import defaultdict
curr_branch = git.Repo('.').active_branch.name
doc_dir = '/home/country/Documents/'
today_date = datetime.date.today()

### Comparing

In [2]:
def code_snippet_1():
    old = RouteStatus.objects.prefetch_related('route', 'delivery_boy').filter(date=datetime.date.today())
    lr = []
    for r in old:
        lr.append(r.route.name)
    return lr

def code_snippet_2():
    old = RouteStatus.objects.filter(date=datetime.date.today()).prefetch_related('route', 'delivery_boy')
    lr = []
    for r in old:
        lr.append(r.route.name)
    return lr

In [70]:
print 'Code Snippet 1 output:',len(code_snippet_1())
print 'Code Snippet 2 output:',len(code_snippet_2())
print '\nComparision Result:'
TestingHelper.compare(100)

Code Snippet 1 output: 1
Code Snippet 2 output: 1

Comparision Result:
Code Snippet 2(1.87559199333) is 3.2% or 1.03 times faster than Code Snippet 1(1.93750095367)


### PlayGround

In [ ]:
aggre = list(BoyProductInventory.objects.using('prod').filter(
            boy_inventory_id__in=boy_inv,product=70).values(
            'product__name','product','boy_inventory__date', 'boy_inventory__boy__center_id',
    'boy_inventory__boy__center__name').annotate(Sum('demand'),
                                                Sum('taken'),Sum('returned'),Sum('leakage'),Sum('sales'),
                                                Sum('other'), Sum('staff')))

In [8]:
BoyInventory.objects.using('prod').filter(date=today_date, taken_at__isnull=False, return_at__isnull=True)

<QuerySet [<BoyInventory: ABDULHAMID WAHID SHAIKH(MAN1908219)>, <BoyInventory: Abdul Razzak(MKE1907083)>, <BoyInventory: Abdul Sayad(MSN1908522)>, <BoyInventory: Altaf Ansari(SDL1908433)>, <BoyInventory: Abhishek Vilas  Sakore(PVS1809029)>, <BoyInventory: Akshay Prahlad Palav(PVS1810135)>, <BoyInventory: Akash Dadasaheb  Nalawade(PVS1809031)>, <BoyInventory: Akshay umesh shinde(PVS1812122)>, <BoyInventory: Akash dayanand jagdale(PVS1904246)>, <BoyInventory: Abhishek Bhaskar Lokhande(NMN1908242)>, <BoyInventory: Akash bose(NDL1904274)>, <BoyInventory: Akhilesh lautanram jayswal  PVS(PVS1908602)>, <BoyInventory: Ajay Kumar S Math(BHL1906270)>, <BoyInventory: Akshay Patwa(MSN1905111)>, <BoyInventory: Akshay Suresh Ingale PNM(PNM1906154)>, <BoyInventory: Amar Nath Jha(SDL1804002)>, <BoyInventory: Abhishek Dadasaheb Bhapkar PVS(PVS1908237)>, <BoyInventory: Abhishek.P(BJP1908020)>, <BoyInventory: Abhishek Dattatray Tarde(PNM1908007)>, <BoyInventory: Altaf Shaikh(MKE1905069)>, '...(remaining 

In [10]:
DeliveryStatus.objects.first().delivery.route.distributioncentre_set.values_list('name',flat=True)

<QuerySet [u'Dwarka']>

In [13]:
Group.objects.values_list('name')

<QuerySet [(u'Super general Admin',), (u'Digicall',), (u'admin',), (u'customercare',), (u'Testing',), (u'General admin',), (u'manager',), (u'moderator',)]>

In [28]:
from django.db.models import Avg, Count
Document.objects.values('status').annotate(Count('status'))

<QuerySet [{'status': u're', 'status__count': 24}, {'status': u've', 'status__count': 305}, {'status': u'pd', 'status__count': 1250}]>

In [39]:
Document.objects.filter(status='pd',file__isnull=True).count()

0

### Roles and Me

In [4]:
roles = list(Group.objects.all().values_list('name', flat=True))

In [5]:
roles

[u'Super general Admin',
 u'Digicall',
 u'admin',
 u'customercare',
 u'Testing',
 u'General admin',
 u'manager',
 u'moderator']

In [31]:
myuser = User.objects.get(id=2881)
for role in roles:
    Group.objects.get(name=role).user_set.remove(myuser) 
myuser.is_superuser=True
myuser.save()

## Old and New Query

In [38]:
# New Query(1.96520900726) is 8.7% or 1.1 times faster than Old Query(2.15243196487)
from django.db.models.functions import TruncDay
old = DeliveryStatus.objects.filter(delivery__deliveryBoy__uid='NOD1809014', delivered=True,
                               date__month=2, date__year=2019
                               ).annotate(day=TruncDay('date')
                                          ).values('day'
                                                   ).annotate(count=Count('id')
                                                              ).filter(count__gt=50)
new = DeliveryStatus.objects.filter(delivery__deliveryBoy__uid='NOD1809014', delivered=True,
                               date__month=2, date__year=2020
                               ).values('date').annotate(count=Count('date')).filter(count__gt=50)

## Create Test Delivery Boy

In [3]:
# Base: CEN1704002(id=1)
def create_test_delivery_boy(name, mobile, base_id=1, roles=['Testing']):
    if not DeliveryBoy.objects.filter(mobile=mobile).exists():
        boy = DeliveryBoy.objects.get(id=base_id)

        boy_cust = Customer.objects.get(id=boy.customer_id)
        boy_cust.pk = None
        boy_cust.id = Customer.objects.latest('id').id + 1
        boy_cust.phone = mobile
        boy_cust.name = name + 'Test'
        boy_cust.save()

        boy_user = User.objects.get(id=boy.user_id)
        boy_user.pk = None
        boy_user.id = User.objects.latest('id').id + 1
        boy_user.username = mobile
        boy_user.save()
        for role in roles:
            Group.objects.get(name=role).user_set.add(boy_user)

        boy.pk = None
        boy.id = DeliveryBoy.objects.latest('id').id + 1
        boy.customer_id = boy_cust.id
        boy.name = name
        boy.acc_holder = name
        boy.father = 'GOD'
        boy.mobile = mobile
        boy.doj = datetime.date.today()
        boy.uid = boy.get_uid()
        boy.user_id = boy_user.id
        boy.save()

        Token.objects.create(user_id=boy.user_id)
        return boy
    return 'Delivery Boy with given mobile number already exists'

In [7]:
boy = create_test_delivery_boy('Maddy', '9354372322')

## Save to excel

### New DB Query

In [41]:
import pandas as pd
from collections import defaultdict
# UID | Name | DOJ | Per Diem Applicable Date | First Date of Present |

db_id_done = {}
query_iter = iter(Attendance.objects.using('prod').filter(boy__doj__year=2019,
                                                          boy__doj__month=8,
                                                          present=True
                                                          ).order_by('date'
                                                                     ).values_list('boy',
                                                                                   'boy__name',
                                                                                   'boy__uid',
                                                                                   'boy__doj',
                                                                                   'date'))

value_dict = defaultdict(list)
for db,name,uid,doj,date in query_iter:
    if not db_id_done.get(db, False):
        value_dict['UID'].append(uid)
        value_dict['Name'].append(name)
        value_dict['DOJ'].append(doj)
        value_dict['First date of present'].append(date)
        per_diem = PerDiem.objects.using('prod').filter(delivery_boy_id=db
                                                  ).order_by('start_date'
                                                             ).first()
        value_dict['Per Diem applicable date'].append(per_diem.start_date)
        db_id_done[db] = True

In [74]:
df = pd.DataFrame.from_dict(value_dict,)
with pd.ExcelWriter(doc_dir+'output.xlsx', engine='xlsxwriter') as writer:  # doctest: +SKIP
     df.to_excel(writer, sheet_name='Sheet_1', index=False)
     # df2.to_excel(writer, sheet_name='Sheet_name_2')

### Demand And Sales

In [ ]:
demand_data = list(DeliveryProduct.objects.using('prod').filter(delivery__date__gte=datetime.date(2019,6,8), 
                                  ).exclude(delivery__route__name__istartswith='app test'
                                           ).values('delivery__date',
                                             'delivery__route__distributioncentre__name',
                                             'product__name',
                                            ).annotate(demand=Sum('quantity')))
demand_df = pd.DataFrame.from_dict(demand_data)

sale_data = list(DeliveryStatusProduct.objects.using('prod').filter(
                delivery_status__delivery__date__gte=datetime.date(2019,6,8), 
                delivery_status__delivered=True
                        ).exclude(delivery_status__delivery__route__name__istartswith='app test'
                                 ).values('delivery_status__delivery__date',
                                 'delivery_status__delivery__route__distributioncentre__name',
                                 'delivery_product__name'
                    ).annotate(sale=Sum('quantity')))
sale_df = pd.DataFrame.from_dict(sale_data)

In [ ]:
demand_df = demand_df[[u'delivery__date', u'delivery__route__distributioncentre__name',
                       u'product__name', u'demand']]
sales_df = sales_df[[u'delivery_status__delivery__date',
                     u'delivery_status__delivery__route__distributioncentre__name',
                     u'delivery_product__name', u'sale']]
demand_df.columns = [u'date', u'distribution_centre_name',u'product_name', u'demand']
sales_df.columns = [u'date', u'distribution_centre_name',u'product_name', u'sales']

In [ ]:
demand_sales_df = pd.merge(demand_df,sales_df,how='outer',
                           on=[u'date', u'distribution_centre_name',u'product_name'])
pd.pivot_table(demand_sales_df,['demand','sales'],
               ['date', u'distribution_centre_name','product_name']
              ).to_excel(doc_dir+'Demand_Sales.xls')

## Create deliveries

Early customer routes ids

In [59]:
ids = list(Delivery.objects.filter(early_customer=True
                                  ).distinct('customer_id'
                                            ).values_list('customer_id', 
                                                          flat=True))
pids = list(Delivery.objects.using('prod').filter(date=datetime.datetime.today(),
                                                  early_customer=True, 
                                                  customer_id__in=ids
                                                 ).values_list('customer_id',
                                                               flat=True)) 
r = list(Route.objects.filter(delivery__customer_id__in=pids).distinct())
for ind, route in enumerate(r):
    print(route.id)
    if ind>5:
        break

In [39]:
# for CEN1704002(ID=1) and GGN1908002(ID=2496)
if curr_branch=='sandbox':
    db_ids = [1, 4900]
else:
    db_ids = [1,2496]
route_ids = [391,390]
# TEST ROUTES
# route_ids = [36,37,38]
from remoteApp.management.commands.test import Command
self = Command()
self.create_deliveries(route_ids)

Aakash Bose NDL
Haider Khan NDL
insert or update on table "daily_deliveryproduct" violates foreign key constraint "daily_deliveryprodu_product_id_bdb7acdc_fk_remoteApp_product_id"
DETAIL:  Key (product_id)=(85) is not present in table "remoteApp_product".



### Assign Deliveres and Cash Pickup

In [40]:
for db_id, route_id in zip(it.cycle(db_ids), route_ids):
    Delivery.objects.filter(date= datetime.date.today(),
                            route_id = route_id
                            ).update(deliveryBoy_id=db_id)
    CashPickup.objects.filter(date=datetime.date.today(),
                              cashpickupstatus__collected_time__isnull=True,
                              route_id=route_id
                              ).update(cash_collector_id=db_id,
                                       high_priority=True)

## Testing Helper

In [4]:
from timeit import timeit
from django.test.utils import CaptureQueriesContext
from django.db import connection
import sqlparse

class TestingHelper():
    @staticmethod
    def print_all(*args):
        for var in args:
            print()
            try:
                print(var.__name__)
            except:
                pass
            try:
                print(var.objects.filter().first().__dict__)
            except:
                try:
                    var = var.__dict__
                except:
                    pass
                if type(var)==dict:
                    for key,val in var.items():
                        print('{key} - {val}'.format(key,val))
                else:
                    print(var)
    @staticmethod
    def compare(num=10000):
        """
        Compares two code snippets with respect to time.

        to compare two code snippets:
            write code snippets in func1 and func2
            call uncomment this function definition and call

        """
        t1 = timeit("code_snippet_1()", setup="from __main__ import code_snippet_1", number=num)
        t2 = timeit("code_snippet_2()", setup="from __main__ import code_snippet_2", number=num)
        
#         t1 = %timeit -n num -o func1()
#         t1 = t1.worst
#         t2 = %timeit -n num -o func2()
#         t2 = t2.worst
        
        per_diff = round((t2-t1) * 100 / t1, 2)
        if per_diff > 0:
            times_diff = round(t2 / t1, 2)
            print('Code Snippet 2({t2}) is {per_diff}% or {times_diff} times slower than Code Snippet 1({t1})'.format(t2=t2,
                                                                                                                      per_diff=per_diff,
                                                                                                                      times_diff=times_diff,                                                                                                                      t1=t1))
        else:
            times_diff = round(t1 / t2, 2)
            print('Code Snippet 2({t2}) is {per_diff}% or {times_diff} times faster than Code Snippet 1({t1})'.format(t2=t2,
                                                                                                                      per_diff=-per_diff,
                                                                                                                      times_diff=times_diff,                                                                                                               t1=t1))
    @staticmethod
    def queries_analyser(func=None,print_sql=True,queryset=None):
        with CaptureQueriesContext(connection) as context:
            if func:
                func()
            elif queryset:
                print(queryset)
        print 'Total Queries:', context.final_queries - context.initial_queries
        if print_sql:
            for query in context.captured_queries:
                try:
                    print("")
                    print(sqlparse.format(query['sql'], reindent=True))
                    print("")
                    print('Execution time: %s'%query['time'])                
                except:
                    print(query)
        return context

## MilkCollection Pandas

In [30]:
def upload_excel(self, request):
    template = 'admin/milkcollection/milkcollectionuser/excel_upload.html'
    if request.method == 'POST':
        form = UserImport(request.POST, request.FILES)
        df = pd.read_excel(request.FILES['fileupload'])
        df.columns = [col.lower().replace(' ','_') for col in df.columns]
        # validation = ImportExcel(request, UserImport, template, self)
        # read_status, content = validation.read_contents()
        try:
            for row in df.itertuples():
                if MilkCollectionUser.objects.filter(mobile=row.mobile).exists():
                    raise Exception('The user with {0} already exists'.format(row.mobile))
                bank, flag = Bank.objects.get_or_create(ifsc=row.bank_ifsc,
                                                        defaults={'bank_name': row.bank_name,
                                                                       'ifsc': row.bank_ifsc})
                bank_details = BankDetails.objects.create(account_no=int(row.account_number), bank=bank, name=row.account_name,
                                                          acc_address=row.account_address)
                try:
                    centre = CollectionCentre.objects.get(centre_id=row.centre_uid)
                except CollectionCentre.DoesNotExist:
                    error_msg = 'Collection Center UID {0} does not exist'.format(row.centre_uid)
                    return  render(request, template,
                                   context={'form': form,
                                            'error': error_msg})

                MilkCollectionUserDefaults = {'name': unicode(row.name, encoding='utf8'),
                                              'centre': centre,
                                              'address': row.address,
                                              'bank_account': bank_details,
                                              'old_id': row._12}
                user, flag = MilkCollectionUser.objects.get_or_create(mobile=row.mobile,
                                                                      defaults=MilkCollectionUserDefaults)
                if row.is_agent == 'YES':
                    CollectionAgent.objects.get_or_create(user=user)
                if row.is_farmer == 'YES':
                    Farmer.objects.get_or_create(user=user)
        except Exception as e:
            return render(request, template, context={'form': form,
                                                      'error': 'Error : {0} Row-{1}'.format(e.message, row.name)})
        return HttpResponseRedirect(reverse('admin:milkcollection_milkcollectionuser_changelist'))
    else:
        return render(request, template, context={'form': UserImport})